# Text Classification using RoBERTa

### Using RoBERTa base and getting AUROC values of ~0.93 with a 100K downsampled/upsampled data set.

In [1]:
#Installs and imports
#!conda create -y -n st python pandas tqdm
#!conda activate st && conda init bash

## for mac
#!conda install -y -n st pytorch torchvision torchaudio -c pytorch-nightly

## for linux
#!conda install -y -n tf pytorch torchvision torchaudio cudatoolkit=11.6 -c pytorch -c conda-forge
#!python3 -m pip install --upgrade transformers simpletransformers wandb

In [2]:
#!pip install simpletransformers

In [3]:
#from transformers import LayoutLMv2Processor
import simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs

# from simpletransformers.classification import ClassificationModel
#from transformers import RobertaForSequenceClassification
import pandas as pd
import logging
import sklearn
import numpy as np

C:\Users\ICS\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\detection\anchor_utils.py:63: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  device: torch.device = torch.device("cpu"),


In [4]:
#Log Results
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

### 2. Import and Transform Data

In [5]:
#Import Train and Test Datasets

df_ham = pd.read_csv('../texts/hamCleanScrubbed.txt', sep='\t', names=['labels','text']).drop_duplicates()
df_spam = pd.read_csv('../texts/spamCleanScrubbed.txt', sep='\t', names=['labels','text']).drop_duplicates()

## Drop header if needed
df_ham = df_ham.iloc[1: , :]
df_spam = df_spam.iloc[1: , :]

print('ham dataset:','\n',df_ham.head(),'\n\n','spam dataset:','\n',df_spam.head())

ham dataset: 
   labels                                               text
1    Ham  11616056 has a WIC appt on {Month} {Day} {Time...
2    Ham  1st of the Holiday Stress Busting Series, Art ...
3    Ham  21417272 has a WIC appt on {Month} {Day} {Time...
4    Ham  2 flu shots for  has an upcoming appt on {Mont...
5    Ham  31018019 has a WIC appt on {Month} {Day} at {T... 

 spam dataset: 
   labels                                               text
1   Spam  {FirstName} has a WIC appt with People's Healt...
2   Spam  {FirstName} has missed a WIC appt on  {Weekday...
3   Spam  {FirstName} missed a WIC appt. Please call {Ph...
4   Spam   {FirstName} has missed a WIC appt on {Weekday...
5   Spam  {FirstName} has a WIC appt with People's Healt...


In [6]:
df_spam

,labels,text
1,Spam,{FirstName} has a WIC appt with People's Healt...
2,Spam,{FirstName} has missed a WIC appt on {Weekday...
3,Spam,{FirstName} missed a WIC appt. Please call {Ph...
4,Spam,{FirstName} has missed a WIC appt on {Weekday...
5,Spam,{FirstName} has a WIC appt with People's Healt...
...,...,...
184,Spam,"You were scheduled for an appt {Weekday}, {Mon..."
185,Spam,"You were scheduled for an appt {Weekday}, {Mon..."
186,Spam,"You were scheduled for an appt {Weekday}, {Mon..."
187,Spam,"You were scheduled for an appt {Weekday}, {Mon..."


In [7]:
df_ham

,labels,text
1,Ham,11616056 has a WIC appt on {Month} {Day} {Time...
2,Ham,"1st of the Holiday Stress Busting Series, Art ..."
3,Ham,21417272 has a WIC appt on {Month} {Day} {Time...
4,Ham,2 flu shots for has an upcoming appt on {Mont...
5,Ham,31018019 has a WIC appt on {Month} {Day} at {T...
...,...,...
260960,Ham,"預約提示：{FirstName}有一個預約在奶票部San Antonio wic于周一, 7..."
260961,Ham,"預約提示：Zoey有一個預約在奶票部San Antonio wic于周二, 11月 05日 ..."
260962,Ham,"預約提示：Zoey有一個預約在奶票部San Antonio wic于周二, 2月 04日 @..."
260963,Ham,"預約提示：{FirstName}有一個預約在奶票部San Antonio wic于周五, 4..."


In [8]:
## convert text to string
df_ham['labels'] = df_ham['labels'].str.lower()
df_spam['labels'] = df_spam['labels'].str.lower()



In [9]:
df_spam

,labels,text
1,spam,{FirstName} has a WIC appt with People's Healt...
2,spam,{FirstName} has missed a WIC appt on {Weekday...
3,spam,{FirstName} missed a WIC appt. Please call {Ph...
4,spam,{FirstName} has missed a WIC appt on {Weekday...
5,spam,{FirstName} has a WIC appt with People's Healt...
...,...,...
184,spam,"You were scheduled for an appt {Weekday}, {Mon..."
185,spam,"You were scheduled for an appt {Weekday}, {Mon..."
186,spam,"You were scheduled for an appt {Weekday}, {Mon..."
187,spam,"You were scheduled for an appt {Weekday}, {Mon..."


In [10]:
df_ham

,labels,text
1,ham,11616056 has a WIC appt on {Month} {Day} {Time...
2,ham,"1st of the Holiday Stress Busting Series, Art ..."
3,ham,21417272 has a WIC appt on {Month} {Day} {Time...
4,ham,2 flu shots for has an upcoming appt on {Mont...
5,ham,31018019 has a WIC appt on {Month} {Day} at {T...
...,...,...
260960,ham,"預約提示：{FirstName}有一個預約在奶票部San Antonio wic于周一, 7..."
260961,ham,"預約提示：Zoey有一個預約在奶票部San Antonio wic于周二, 11月 05日 ..."
260962,ham,"預約提示：Zoey有一個預約在奶票部San Antonio wic于周二, 2月 04日 @..."
260963,ham,"預約提示：{FirstName}有一個預約在奶票部San Antonio wic于周五, 4..."


In [11]:
## change ham to 0 and spam to 1
df_ham['labels'] = df_ham['labels'].replace({'ham':'0', 'spam':'1'})
df_spam['labels'] = df_spam['labels'].replace({'ham':'0', 'spam':'1'})



In [12]:
df_spam

,labels,text
1,1,{FirstName} has a WIC appt with People's Healt...
2,1,{FirstName} has missed a WIC appt on {Weekday...
3,1,{FirstName} missed a WIC appt. Please call {Ph...
4,1,{FirstName} has missed a WIC appt on {Weekday...
5,1,{FirstName} has a WIC appt with People's Healt...
...,...,...
184,1,"You were scheduled for an appt {Weekday}, {Mon..."
185,1,"You were scheduled for an appt {Weekday}, {Mon..."
186,1,"You were scheduled for an appt {Weekday}, {Mon..."
187,1,"You were scheduled for an appt {Weekday}, {Mon..."


In [13]:
df_ham

,labels,text
1,0,11616056 has a WIC appt on {Month} {Day} {Time...
2,0,"1st of the Holiday Stress Busting Series, Art ..."
3,0,21417272 has a WIC appt on {Month} {Day} {Time...
4,0,2 flu shots for has an upcoming appt on {Mont...
5,0,31018019 has a WIC appt on {Month} {Day} at {T...
...,...,...
260960,0,"預約提示：{FirstName}有一個預約在奶票部San Antonio wic于周一, 7..."
260961,0,"預約提示：Zoey有一個預約在奶票部San Antonio wic于周二, 11月 05日 ..."
260962,0,"預約提示：Zoey有一個預約在奶票部San Antonio wic于周二, 2月 04日 @..."
260963,0,"預約提示：{FirstName}有一個預約在奶票部San Antonio wic于周五, 4..."


In [14]:
## confirm there's no ham in spam and vice versa
#df_ham = df_ham[df_ham['labels'] == 0]
#df_spam = df_spam[df_spam['labels'] == 1]



In [15]:
df_ham

,labels,text
1,0,11616056 has a WIC appt on {Month} {Day} {Time...
2,0,"1st of the Holiday Stress Busting Series, Art ..."
3,0,21417272 has a WIC appt on {Month} {Day} {Time...
4,0,2 flu shots for has an upcoming appt on {Mont...
5,0,31018019 has a WIC appt on {Month} {Day} at {T...
...,...,...
260960,0,"預約提示：{FirstName}有一個預約在奶票部San Antonio wic于周一, 7..."
260961,0,"預約提示：Zoey有一個預約在奶票部San Antonio wic于周二, 11月 05日 ..."
260962,0,"預約提示：Zoey有一個預約在奶票部San Antonio wic于周二, 2月 04日 @..."
260963,0,"預約提示：{FirstName}有一個預約在奶票部San Antonio wic于周五, 4..."


In [16]:
df_spam

,labels,text
1,1,{FirstName} has a WIC appt with People's Healt...
2,1,{FirstName} has missed a WIC appt on {Weekday...
3,1,{FirstName} missed a WIC appt. Please call {Ph...
4,1,{FirstName} has missed a WIC appt on {Weekday...
5,1,{FirstName} has a WIC appt with People's Healt...
...,...,...
184,1,"You were scheduled for an appt {Weekday}, {Mon..."
185,1,"You were scheduled for an appt {Weekday}, {Mon..."
186,1,"You were scheduled for an appt {Weekday}, {Mon..."
187,1,"You were scheduled for an appt {Weekday}, {Mon..."


In [17]:
## Resample data
from sklearn.utils import resample
## Downsample majority class
df_ham_downsampled = resample(df_ham, 
                                replace=True,   ##replace=False when downsampling, True when upsampling
                                n_samples=50)


In [18]:
#Upsample minority class
df_spam_upsampled = resample(df_spam, 
                                replace=True,     
                                n_samples=50)

# Combine minority class with downsampled majority class
df_up_down_sampled = pd.concat([df_ham_downsampled, df_spam_upsampled]).drop_duplicates()

## create train/test split
train = np.random.rand(len(df_up_down_sampled)) < 0.75
df_train = df_up_down_sampled[train]

df_train.reset_index(drop=True, inplace=True)
df_train.loc[:, 'id'] = df_train.index + 1
df_train['id'] = df_train['id'].astype(pd.Int64Dtype())
df_train = df_train[['id','text','labels']]
df_train.to_csv('train.csv', index=False)

df_test = df_up_down_sampled[~train]
df_test.reset_index(drop=True, inplace=True)
df_test.loc[:,'id'] = len(df_train.index) + df_test.index + 1
df_test['id'] = df_test.loc[:,'id'].astype(pd.Int64Dtype())
df_test = df_test[['id','text','labels']]
df_test.to_csv('test.csv', index=False)

## print train/test data sizes
print("training rows: ",len(df_train.index))
print("testing rows: ",len(df_test.index))

## drop individual tables to free memory
del [[df_ham,df_spam,df_ham_downsampled,df_spam_upsampled,df_up_down_sampled]]

C:\Users\ICS\AppData\Local\Temp\ipykernel_9688\3686834458.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.loc[:, 'id'] = df_train.index + 1
C:\Users\ICS\AppData\Local\Temp\ipykernel_9688\3686834458.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['id'] = df_train['id'].astype(pd.Int64Dtype())
C:\Users\ICS\AppData\Local\Temp\ipykernel_9688\3686834458.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

training rows:  77
testing rows:  18


In [19]:
## Drop ID columns for training/testing
df_train.drop(columns = 'id')
df_test.drop(columns = 'id')
print(df_train.head(),df_test.head())

   id                                               text labels
0   1  {FirstName} has a WIC appt on {Month} {Day} {T...      0
1   2  Allison Castro A partir del 1 de {Month} de 20...      0
2   3  This is the Morris Heights Center WIC program ...      0
3   4  Avery has a WIC appt on {Weekday}, {Month} {Da...      0
4   5  {FirstName} {LastName} Marion has a WIC appt o...      0    id                                               text labels
0  78  {FirstName} {LastName} Jarred has a WIC appt o...      0
1  79  Mandalyn has an appt with HCA on {Month} {Day}...      0
2  80  This is a reminder for Jasmine you have a WIC ...      0
3  81  This is Sacaton {Clinic} with a reminder to ve...      0
4  82  Saludos de WIC, su cita de WIC fue el viernes ...      0


In [20]:
## Check dataframe type for model
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      77 non-null     Int64 
 1   text    76 non-null     object
 2   labels  77 non-null     object
dtypes: Int64(1), object(2)
memory usage: 2.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18 non-null     Int64 
 1   text    18 non-null     object
 2   labels  18 non-null     object
dtypes: Int64(1), object(2)
memory usage: 578.0+ bytes


### 3. Create Classification Model

In [21]:
#Set and create the classification model we want to use

model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.learning_rate = 2e-5
model_args.train_batch_size = 16
model_args.eval_batch_size = 16
model_args.manual_seed = 42
model_args.optimized = "AdamW"
model_args.adam_epsilon = 1e-8
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True




In [22]:
#model = ClassificationModel('roberta', 'mariagrandury/roberta-base-finetuned-sms-spam-detection', num_labels=2,use_cuda=False, weight=[1 , len(df_ham) / len(df_spam_upsampled)], args=model_args)
model = ClassificationModel('roberta', 'mariagrandury/roberta-base-finetuned-sms-spam-detection',use_cuda=False, num_labels=2, weight=[1 , 1], args=model_args)
#model = ClassificationModel('roberta', 'roberta-base', use_cuda=False, args={'reprocess_input_data': True, 'overwrite_output_dir': True})



In [ ]:
#Train the classification model using training data
model.train_model(df_train)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/77 [00:00<?, ?it/s]

### 4. Evaluate the Model

- Our trained model was able to produce an accuracy of 99.3%. It accurately identified 656 spam messages, accurately identified 4311 non-spam messages, falsely identified 16 messages as spam, and 17 messages as non-spam.

In [ ]:
#Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(df_test, acc=sklearn.metrics.accuracy_score)

### 5. Make Predictions on Test Data and Save to CSV

In [ ]:
#Run the model on test data
print(model.config.id2label)

predictions, raw_outputs = model.predict(df_test['text'].astype(str).values.tolist())

print(predictions, raw_outputs)

from scipy.special import softmax
probabilities = softmax(raw_outputs)

In [ ]:
#Export Predictions
preds = pd.DataFrame({'id': df_test.index, 'predicted': predictions})
preds.head()
preds.to_csv('predictions.csv', index=False)